In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

trainData = pd.read_csv('/kaggle/input/titanic/train.csv')
testData = pd.read_csv('/kaggle/input/titanic/test.csv')


# Setup the figure of matplotlib
x, y = plt.subplots(figsize=(11, 9))

# Compute a diverging color map
cmap = sns.diverging_palette(200, 30, as_cmap=True)

# Drawing the heatmap with the correct aspect ratio
sns.heatmap(trainData.corr(),  cmap=cmap, center=0,
            square=True, linewidths=0.6, cbar_kws={"shrink": 0.6})
print(trainData.corr())

Here we import the .csv files and convert them to dataframes to print and visualize the correlation matrix. The features with the highest correlation with "Survived" are Pclass, Sex, Age and Fare.

In [ ]:

Features = ["Pclass", "Sex", "Age", "Fare","Survived"]
temp = trainData[Features]
meanValue = temp['Age'].mean()
temp['Age'] = temp['Age'].fillna(meanValue)
meanValue = temp['Fare'].mean()
temp['Fare'] = temp['Fare'].fillna(meanValue)

temp["Sex"].replace('female', 0, inplace=True)
temp["Sex"].replace('male', 1, inplace=True)

In this part, we only use the features decided by correlation as well as adding the sex feature as it wasn't in correlation. We need to fill the null values in Age and fare features as logistic regression cannot work with null value.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(temp, temp["Sex"]):
    strat_train_set = temp.loc[train_index]
    strat_test_set = temp.loc[test_index]

In this part we split the data by shuffle split to maintain the percentages of males and females in the data set.

In [ ]:
features = ["Pclass", "Sex", "Age", "Fare"]  
x_train = strat_train_set[features]
y_train = strat_train_set["Survived"]
x_val = strat_test_set[features]
y_val = strat_test_set["Survived"]

Here we prepare the model inputs such as x_train that contains the features calues needed for training the model and y_train containing the outputs of survived with correspondence with x_train features.

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(x_train, y_train)
scores = logistic.score(x_val, y_val)
print(scores)

In [ ]:
Test = testData[features]
meanValue = Test['Age'].mean()
Test['Age'] = Test['Age'].fillna(meanValue)
meanValue = Test['Fare'].mean()
Test['Fare'] = Test['Fare'].fillna(meanValue)

Test["Sex"].replace('female', 0, inplace=True)
Test["Sex"].replace('male', 1, inplace=True)
Test_preds = logistic.predict(Test)

output = pd.DataFrame({'PassengerId': testData.PassengerId,
                       'Survived': Test_preds})
print(output)
output.to_csv('submission.csv', index=False)

We submit the preprocessing to the test data and use it to predict who survived to be submitted. We turned it to dataframe then to .csv file